In [1]:
#split apply combine
#dividing dataset into independent groups
#using groupby
#most common use of groupby object is to perform aggregation
#determining average arrival delay of each flight

import pandas as pd
import numpy as np 

flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [2]:
#grouping flights by airline and then calling the aggregate method
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head()
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [10]:
'''
agg with dictionary
agg with list
directly applying appropriate agg function without agg()
'''

#finding number of cancelled flights for every airline per weekday
#finding number and percentage of cancelled and diverted flights for every airline per weekday
#for each origin and destination finding total number of flights the number and percentage of cancelled flights and average and variance of airtime

flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg(['sum', 'mean']).head(7)

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
        6              21  0.018667        9  0.008000
        7              29  0.021837        1  0.000753

In [12]:
#performing aggregation with groupby() to create a df with MultiIndex for rows and cols

#find total and avg miles flown and max and min arrival delay for each airline for each weekday
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY']).agg({'DIST':['sum', 'mean'], 'ARR_DELAY': ['min', 'max']}).astype(int)
airline_info.head(7)

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
        6        1265340  1124       -50  858
        7        1461906  1100       -49  626

In [8]:
#multiindexing with 2 levels
#squashing the levels..!

level0 = airline_info.columns.get_level_values(0)
level1 = airline_info.columns.get_level_values(1)
print(level0)
print(level1)
#see table on page 226 of book to get better idea of squashing!!

#we can use reset_index() to set the index to single level and return the df

#aggregating functions - bottom level
#aggregating columns - top level

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')
Index(['sum', 'mean', 'min', 'max'], dtype='object')


In [6]:
#by default - end of groupby operation pandas puts all grouping columns in the index
#as_index parameter can be set to false to avoid this

flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean').round(0)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0
7,MQ,404.0
8,NK,1047.0
9,OO,511.0


In [14]:
#using college data set to calc mean and std dev of student pop per state
college = pd.read_csv('data/college.csv')
college.groupby('STABBR')['UGDS'].agg(['mean', 'std']).round(0).head()

#here the output is not what is desired.
#we aren't looking for mean and SD of entire group, but max number of SD away from mean for any one institution!

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0


In [26]:
#Pandas doesnt provide func to calculate max number of SD away from mean

def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()
college.groupby('STABBR')['UGDS'].agg(max_deviation).round(1).head()

'''
Looking ahead at step 3, you will notice that the function name is placed
inside the agg method without directly being called. Nowhere is the parameter s explicitly
passed to max_deviation . Instead, pandas implicitly passes the UGDS column as a Series to
max_deviation
'''

college['UGDS'].describe()

count      6874.000000
mean       2356.837940
std        5474.275871
min           0.000000
25%         117.000000
50%         412.500000
75%        1929.500000
max      151558.000000
Name: UGDS, dtype: float64